# Structured Data Assignment

# Problem Statement 1

The development of drugs is critical in providing therapeutic options for patients suffering from chronic and terminal illnesses. “Target Drug”, in particular, is designed to enhance the patient's health and well-being without causing
dependence on other medications that could potentially lead to severe and life-threatening side effects. These drugs are specifically tailored to treat a particular disease or condition, offering a more focused and effective approach to treatment,
while minimising the risk of harmful reactions. The objective in this assignment is to develop a predictive model which will predict whether a patient will be eligible*** for “Target Drug” or not in next 30 days. Knowing if the patient is eligible or not will help physician treating the patient make informed decision on the which treatments to give.

In [28]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
# to load the parquet file first we need to install pyarrow package
! pip install pyarrow

In [4]:
# Loading the training dataset
train_data_df = pd.read_parquet('Train.parquet')

In [5]:
# to view the data 
train_data_df

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1
...,...,...,...
29080886,a0ee9f75-1c7c-11ec-94c7-16262ee38c7f,2018-07-06,DRUG_TYPE_6
29080897,a0ee1284-1c7c-11ec-a3d5-16262ee38c7f,2017-12-29,DRUG_TYPE_6
29080900,a0ee9b26-1c7c-11ec-8a40-16262ee38c7f,2018-10-18,DRUG_TYPE_10
29080903,a0ee1a92-1c7c-11ec-8341-16262ee38c7f,2015-09-18,DRUG_TYPE_6


In [6]:
# To get the first 5 rows
train_data_df.head()

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1


In [7]:
# to get the last 5 rows
train_data_df.tail()

,Patient-Uid,Date,Incident
29080886,a0ee9f75-1c7c-11ec-94c7-16262ee38c7f,2018-07-06,DRUG_TYPE_6
29080897,a0ee1284-1c7c-11ec-a3d5-16262ee38c7f,2017-12-29,DRUG_TYPE_6
29080900,a0ee9b26-1c7c-11ec-8a40-16262ee38c7f,2018-10-18,DRUG_TYPE_10
29080903,a0ee1a92-1c7c-11ec-8341-16262ee38c7f,2015-09-18,DRUG_TYPE_6
29080911,a0ee146e-1c7c-11ec-baee-16262ee38c7f,2018-10-05,DRUG_TYPE_1


In [8]:
# to get the shape of the dataset
train_data_df.shape

(3220868, 3)

In [9]:
# to get the basic information of the dataset
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220868 entries, 0 to 29080911
Data columns (total 3 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Patient-Uid  object        
 1   Date         datetime64[ns]
 2   Incident     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 98.3+ MB


In [11]:
# to get the unique values
train_data_df.nunique()

Patient-Uid    27033
Date            1977
Incident          57
dtype: int64

In [12]:
# to get the feature name
train_data_df.columns

Index(['Patient-Uid', 'Date', 'Incident'], dtype='object')

In [14]:
# to get the data types
train_data_df.dtypes

Patient-Uid            object
Date           datetime64[ns]
Incident               object
dtype: object

In [15]:
# to check if there is any null values present in the dataset
train_data_df.isnull().sum()

Patient-Uid    0
Date           0
Incident       0
dtype: int64

Here in this dataset, there is no null value is present.

In [17]:
# to check if there is any duplicate values
train_data_df.duplicated().sum()

35571

There are 35571 rows are duplicate values in this dataset.

In [18]:
# to drop the duplicate values
train_data_df.drop_duplicates(inplace = True)

In [19]:
# to check if the values are dropped, we can confirm with the shape function
train_data_df.shape

(3185297, 3)

Here, it is clear that the duplicate values are dropped.

Now we need to seperate the positive and negative sets as mentioned in the problem statement here as mentioned in the problem statement, the patients who have taken the target drug are said to be positive one's.

In [22]:
# to get the positive set
positive_set = train_data_df[train_data_df['Incident'] == 'TARGET DRUG']
positive_set

,Patient-Uid,Date,Incident
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG
...,...,...,...
29074998,a0ef2b6d-1c7c-11ec-9172-16262ee38c7f,2018-10-12,TARGET DRUG
29075105,a0ebe423-1c7c-11ec-a5e0-16262ee38c7f,2019-07-02,TARGET DRUG
29075494,a0ebc713-1c7c-11ec-bd53-16262ee38c7f,2019-05-21,TARGET DRUG
29080031,a0ee1bdb-1c7c-11ec-90ba-16262ee38c7f,2018-06-07,TARGET DRUG


In [23]:
# to get the shape of positive set
positive_set.shape

(67218, 3)

Next we need to find the negative set, here patients who have not taken the target drug are said to be the negative one's.  

In [25]:
# to get the negative set
negative_set = train_data_df[~train_data_df['Patient-Uid'].isin(positive_set['Patient-Uid'])]
negative_set = negative_set.groupby('Patient-Uid').tail(1)
negative_set

,Patient-Uid,Date,Incident
1560892,a0e3a8c0-1c7c-11ec-98c2-16262ee38c7f,2018-05-06,PRIMARY_DIAGNOSIS
1620903,a0dd6a3f-1c7c-11ec-9b86-16262ee38c7f,2015-04-07,SYMPTOM_TYPE_0
1629044,a0e48a75-1c7c-11ec-8c5f-16262ee38c7f,2018-08-22,DRUG_TYPE_6
1942882,a0e3cf61-1c7c-11ec-8098-16262ee38c7f,2018-08-21,DRUG_TYPE_2
1975541,a0e91a8c-1c7c-11ec-acc2-16262ee38c7f,2020-04-15,PRIMARY_DIAGNOSIS
...,...,...,...
3256795,a0e045a1-1c7c-11ec-8014-16262ee38c7f,2020-07-10,PRIMARY_DIAGNOSIS
3256799,a0e67e2a-1c7c-11ec-b805-16262ee38c7f,2015-12-16,PRIMARY_DIAGNOSIS
3256800,a0dec400-1c7c-11ec-80df-16262ee38c7f,2019-08-06,PRIMARY_DIAGNOSIS
3256804,a0e09919-1c7c-11ec-9e7d-16262ee38c7f,2017-02-19,DRUG_TYPE_6


In [26]:
# to get the shape of the negative set
negative_set.shape


(17659, 3)

We need to do some feature engineering process in order to get a best model. So as a first step we are going to calculate the count of previous prescriptions within specific time interval based on the frequency and as a second step we are going to find the difference between the most recent prescription and the prediction date.

In [29]:
# to get the count of previous prescriptions within specific time intervals
positive_set['Prescription_Count'] = positive_set.groupby('Patient-Uid')['Date'].cumcount()
negative_set['Prescription_Count'] = negative_set.groupby('Patient-Uid')['Date'].cumcount()

In [30]:
# to get the difference between the most recent prescription and the prediction date.
prediction_date = pd.to_datetime('today') + pd.DateOffset(days=30)
positive_set['Time_Difference'] = (prediction_date - positive_set.groupby('Patient-Uid')['Date'].transform('max')).dt.days
negative_set['Time_Difference'] = (prediction_date - negative_set.groupby('Patient-Uid')['Date'].transform('max')).dt.days

In [31]:
# concatenating the positive and negative sets
new_train_data_df = pd.concat([positive_set, negative_set])

In [32]:
# to view the new train data
new_train_data_df

,Patient-Uid,Date,Incident,Prescription_Count,Time_Difference
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG,0,1058
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG,0,1312
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG,0,1337
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG,0,1042
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG,0,1058
...,...,...,...,...,...
3256795,a0e045a1-1c7c-11ec-8014-16262ee38c7f,2020-07-10,PRIMARY_DIAGNOSIS,0,1092
3256799,a0e67e2a-1c7c-11ec-b805-16262ee38c7f,2015-12-16,PRIMARY_DIAGNOSIS,0,2760
3256800,a0dec400-1c7c-11ec-80df-16262ee38c7f,2019-08-06,PRIMARY_DIAGNOSIS,0,1431
3256804,a0e09919-1c7c-11ec-9e7d-16262ee38c7f,2017-02-19,DRUG_TYPE_6,0,2329


Now we need train a model.

In [34]:
# to split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new_train_data_df[['Prescription_Count', 'Time_Difference']], new_train_data_df['Incident'] == 'TARGET DRUG')

In [35]:
# we are going to train a logistic regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [42]:
# to make predictions on the testing dataset
y_pred = logistic_model.predict(X_test)

In [43]:
# to evaluate the model
conf_matrix_report = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print(conf_matrix_report)
print(class_report)

[[ 4331     0]
 [    0 16889]]
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      4331
        True       1.00      1.00      1.00     16889

    accuracy                           1.00     21220
   macro avg       1.00      1.00      1.00     21220
weighted avg       1.00      1.00      1.00     21220



Now we are going to use our model to predict the test data set which was provided sepeartely.

We need to follow the same preprocessing the techniques which was uesd in the train dataset

In [44]:
# to load the test dataset
test_data_df = pd.read_parquet('Test.parquet')

In [45]:
# to view the data 
test_data_df

,Patient-Uid,Date,Incident
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2016-12-08,SYMPTOM_TYPE_0
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-10-17,DRUG_TYPE_0
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-12-01,DRUG_TYPE_2
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-12-05,DRUG_TYPE_1
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-11-04,SYMPTOM_TYPE_0
...,...,...,...
1372854,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-05-11,DRUG_TYPE_13
1372856,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2018-08-22,DRUG_TYPE_2
1372857,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-02-04,DRUG_TYPE_2
1372858,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-09-25,DRUG_TYPE_8


In [48]:
# To get the first 5 rows
test_data_df.head()

,Patient-Uid,Date,Incident
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2016-12-08,SYMPTOM_TYPE_0
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-10-17,DRUG_TYPE_0
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-12-01,DRUG_TYPE_2
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-12-05,DRUG_TYPE_1
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-11-04,SYMPTOM_TYPE_0


In [47]:
# to get the last 5 rows
test_data_df.tail()

,Patient-Uid,Date,Incident
1372854,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-05-11,DRUG_TYPE_13
1372856,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2018-08-22,DRUG_TYPE_2
1372857,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-02-04,DRUG_TYPE_2
1372858,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-09-25,DRUG_TYPE_8
1372859,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-05-19,DRUG_TYPE_7


In [49]:
# to get the shape of the dataset
test_data_df.shape

(1065524, 3)

In [50]:
# to get the basic information of the dataset
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1065524 entries, 0 to 1372859
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Patient-Uid  1065524 non-null  object        
 1   Date         1065524 non-null  datetime64[ns]
 2   Incident     1065524 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 32.5+ MB


In [51]:
# to get the unique values
test_data_df.nunique()

Patient-Uid    11482
Date            1947
Incident          55
dtype: int64

In [52]:
# to get the feature name
test_data_df.columns

Index(['Patient-Uid', 'Date', 'Incident'], dtype='object')

In [53]:
# to check if there is any null values present in the dataset
test_data_df.isnull().sum()

Patient-Uid    0
Date           0
Incident       0
dtype: int64

In [54]:
# to check if there is any duplicate values
test_data_df.duplicated().sum()

12100

In [55]:
# to drop the duplicate values
train_data_df.drop_duplicates(inplace = True)

In [60]:
# feature engineering process to the test dataset
test_data_df['Prescription_Count'] = test_data_df.groupby('Patient-Uid')['Date'].cumcount()
test_data_df['Time_Difference'] = (prediction_date - test_data_df.groupby('Patient-Uid')['Date'].transform(max)).dt.days

In [61]:
# to predict the test dataset
test_data_prediction = logistic_model.predict(test_data_df[['Prescription_Count', 'Time_Difference']])

In [62]:
# to get the test_data_prediction
test_data_prediction

array([False,  True,  True, ...,  True,  True,  True])

In [63]:
# to calculate the f1 score
F1_score = f1_score(y_test, y_pred)
print('F1 Score = ',F1_score)

F1 Score =  1.0


In [75]:
# to create final submission file
Final_submission = pd.DataFrame({'Patient-Uid': test_data_df['Patient-Uid'], 'Prediction': test_data_prediction})
Final_submission.to_csv('Final_submission.csv', index = False)

In [76]:
# to view the datas in the final submission dataset
final_submission_df = pd.read_csv('Final_submission.csv')
final_submission_df

,Patient-Uid,Prediction
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,False
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,True
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,True
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,True
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,True
...,...,...
1065519,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,True
1065520,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,True
1065521,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,True
1065522,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,True


Here comes an end. Our model has a maximum f1 score of 1 and we used our model to predict values for the provided test data. 